In [1]:
%pylab inline

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import boxcox
from sklearn import metrics
from xgboost import XGBRegressor
from pandas.tseries.holiday import get_calendar, USFederalHolidayCalendar
from IPython.display import clear_output
import pickle
import xgboost
print xgboost.__version__

Populating the interactive namespace from numpy and matplotlib
0.72


Читаем подготовленные данные для сто двух ячеек регионов с каггл. Добавлены новые данные - средняя длина поездки, среднее количество пассажиров, средняя стоимость поездки (без налогов и платных дорог)

In [2]:
#https://github.com/nlavri/mipt_ya_ml/blob/master/data/stats_kaggle_2015-12_2016-06_ext.dat
data = pd.read_pickle('TaxiData/stats_kaggle_2015-12_2016-06_ext.dat')
print data.shape
data.head()

(521322, 8)


,region,trip_distance_mean,tolls_amount_mean,tip_amount_mean,fare_amount_mean,count,passenger_mean,duration
tpep_pickup_datetime,,,,,,,,
2015-12-01,1075,4.802105,0.874737,3.553158,16.052632,19.0,1.789474,00:13:41.263157
2015-12-01,1076,5.043721,0.870930,3.075814,17.662791,43.0,1.418605,00:15:07.209302
2015-12-01,1077,3.607692,0.213077,2.975769,12.769231,26.0,2.076923,00:10:46.923076
2015-12-01,1125,4.318947,0.398553,2.613289,17.203947,76.0,1.565789,00:13:38.013157
2015-12-01,1126,4.824725,0.243516,2.555604,16.362637,91.0,1.659341,00:30:16.164835


Составим признаки для регресии по каждому региону. Данные каждого региона будем помещать в отдельный фрейм, а потом объединим в один общий, чтобы соответствовать постановке задачи и создать шесть моделей на все регионы сразу, а не на каждый регион по отдельности. 

In [3]:
def getDayPart(date_time):
   
    if (date_time.weekday() == 6 or date_time.weekday() == 5):
        if 4 <= date_time.hour and date_time.hour <= 8:
            return 0;
        if 9 <= date_time.hour and date_time.hour <= 18:
            return 1;
        if 19 <= date_time.hour or date_time.hour <= 5:
            return 0;
        
    else:
        if 1 <= date_time.hour and date_time.hour <= 6:
            return 0;
        if 7 <= date_time.hour and date_time.hour <= 18:
            return 1;
        if 19 <= date_time.hour and date_time.hour <= 23:
            return 2;
        if 23 < date_time.hour or date_time.hour == 0:
            return 0;

nModels = 6

K = 40

start_time = pd.to_datetime('2016-01-01 00:00:00')
end_time = pd.to_datetime('2016-06-30 17:00:00')

frames = []

for reg in data.region.unique():
    clear_output(wait=True)
    print reg
    sample_data = data[data.region == reg]
    
    #fix 2016-06-30 23:00:00
    sample_data['count'][pd.to_datetime('2016-06-30 23:00:00')] = sample_data['count'][pd.to_datetime('2016-06-30 22:00:00')]
    
    frame = pd.DataFrame()

    cntData = sample_data[start_time:end_time]['count']
    frame['data'] = cntData
    frame['data_bc'] = boxcox(cntData + cntData.mean())[0]
    
    frame['passenger_mean'] = sample_data[start_time:end_time]['passenger_mean']
    frame['trip_distance_mean'] = sample_data[start_time:end_time]['trip_distance_mean']
    frame['duration'] = sample_data[start_time:end_time]['duration']
    
    frame['fare_amount_mean'] = sample_data[start_time:end_time]['fare_amount_mean']
    frame['tip_amount_mean'] = sample_data[start_time:end_time]['tip_amount_mean']
    frame['tolls_amount_mean'] = sample_data[start_time:end_time]['tolls_amount_mean']
    
    frame['region'] = [reg] * frame.shape[0]
    
    for k in range(1, K + 1):
        frame.loc[:,'s'+ str(k)] = np.sin(np.arange(frame.shape[0]) * 2 * np.pi * k / 168)
        frame.loc[:,'c'+ str(k)] = np.cos(np.arange(frame.shape[0]) * 2 * np.pi * k / 168)

    frame.loc[:, 'day_part'] = frame.index.map(getDayPart)
    frame.loc[:, 'day'] = frame.index.map(lambda x: x.day)
    frame.loc[:, 'month'] = frame.index.map(lambda x: x.month)
    frame.loc[:, 'weekday'] = frame.index.map(lambda x: x.weekday())

        
    Khist = 48

    for k in range(1, Khist + 1):
        t_delta = datetime.timedelta(hours=k)
        historyData = sample_data['count'][start_time - t_delta:end_time - t_delta]
        frame.loc[:, 'data_history_' + str(k)] = historyData.values

    for shift in range(1, nModels + 1):
        delta = datetime.timedelta(hours=shift)
        frame['target_' + str(shift)] = sample_data['count'][start_time + delta:end_time + delta].values

    frames.append(frame)

2168


In [4]:
allData = pd.concat(frames)
allData = allData.sort_index()
print allData.shape

(444924, 147)


In [5]:
allData['duration'] = allData['duration'].dt.total_seconds().astype(int)

Обучим модели на данных до конца апреля включительно (последнее Т это '2016-04-30 17:00:00' соответственно последний отклик для T+6 это отклик на '2016-04-30 23:00:00'. Используем XGBRegressor. 
Для теста используем уже май.

In [6]:
train_from = pd.to_datetime('2016-02-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

x_names = []

x_names.append('region')

x_names.append('passenger_mean')
x_names.append('trip_distance_mean')
x_names.append('duration')
    
x_names.append('fare_amount_mean')
x_names.append('tip_amount_mean')
x_names.append('tolls_amount_mean')
    

for k in range(1, K + 1):
    x_names.append('s'+ str(k))
    x_names.append('c'+ str(k))

x_names.append('data')    
x_names.append('data_bc')    

x_names.append('day_part')
x_names.append('day')
x_names.append('month')
x_names.append('weekday')

for k in range(1, Khist + 1):
    x_names.append('data_history_' + str(k))
            
test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')


Подбираем learning rate

In [27]:
learning_rates = np.linspace(0.21, 0.55, 50)

topRates = {}        
for shift in range(1, nModels + 1):
    bestLearning_rate = 0
    bestMae = sys.maxint

    for learning_rate in learning_rates:

        print 'shift', shift, 'learning_rate is ', learning_rate, 
        
        m = XGBRegressor(#n_jobs=8, 
                         tree_method='gpu_hist',
                         learning_rate=learning_rate)

        trainF = allData[train_from : train_to]
        fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

        mae = metrics.mean_absolute_error(allData[test_from : test_to]['target_' + str(shift)],
                                          fitted.predict(allData[test_from : test_to][x_names]))
        
        if mae < bestMae:
            bestMae = mae
            bestLearning_rate = learning_rate
        
        print 'MAE',  mae

    topRates[shift] = bestLearning_rate
    print 'shift', shift, 'best learning_rate is ', bestLearning_rate, 'MAE', bestMae


shift 1 learning_rate is  0.21 MAE 18.924788258398273
shift 1 learning_rate is  0.21693877551020407 MAE 19.024596362192284
shift 1 learning_rate is  0.22387755102040816 MAE 18.99453378957293
shift 1 learning_rate is  0.23081632653061224 MAE 18.975460999990823
shift 1 learning_rate is  0.23775510204081632 MAE 18.960811511929897
shift 1 learning_rate is  0.2446938775510204 MAE 18.96925106076007
shift 1 learning_rate is  0.2516326530612245 MAE 19.088149951992655
shift 1 learning_rate is  0.25857142857142856 MAE 19.018633657915903
shift 1 learning_rate is  0.2655102040816327 MAE 18.941503607742682
shift 1 learning_rate is  0.2724489795918367 MAE 18.93504491527714
shift 1 learning_rate is  0.27938775510204084 MAE 19.09248414218146
shift 1 learning_rate is  0.2863265306122449 MAE 19.120767429231027
shift 1 learning_rate is  0.293265306122449 MAE 19.10786899325132
shift 1 learning_rate is  0.3002040816326531 MAE 19.067335570075453
shift 1 learning_rate is  0.30714285714285716 MAE 18.998717593

shift 3 learning_rate is  0.34877551020408165 MAE 24.742985196562444
shift 3 learning_rate is  0.35571428571428576 MAE 24.776601857594724
shift 3 learning_rate is  0.3626530612244898 MAE 24.898543561991747
shift 3 learning_rate is  0.36959183673469387 MAE 24.780759178872398
shift 3 learning_rate is  0.37653061224489803 MAE 24.745785319255024
shift 3 learning_rate is  0.3834693877551021 MAE 25.022777437453275
shift 3 learning_rate is  0.39040816326530614 MAE 24.79157634534529
shift 3 learning_rate is  0.39734693877551025 MAE 25.02523628989969
shift 3 learning_rate is  0.40428571428571436 MAE 24.787421067348177
shift 3 learning_rate is  0.4112244897959184 MAE 24.82720225259273
shift 3 learning_rate is  0.41816326530612247 MAE 24.960000637837165
shift 3 learning_rate is  0.4251020408163266 MAE 24.87435277031242
shift 3 learning_rate is  0.4320408163265307 MAE 24.75975932830121
shift 3 learning_rate is  0.43897959183673474 MAE 25.017059809261834
shift 3 learning_rate is  0.4459183673469388

In [28]:
topRates

{1: 0.41816326530612247,
 2: 0.2863265306122449,
 3: 0.3002040816326531,
 4: 0.37653061224489803,
 5: 0.36959183673469387,
 6: 0.35571428571428576}

In [7]:
topRates = {1: 0.41816326530612247,
 2: 0.2863265306122449,
 3: 0.3002040816326531,
 4: 0.37653061224489803,
 5: 0.36959183673469387,
 6: 0.35571428571428576}

Подберем n_estimators

In [29]:
estimators = np.arange(100, 500, 100)

topEstimators = {}        
for shift in range(1, nModels + 1):
    bestEstimator = 0
    bestMae = sys.maxint

    for estimator in estimators:

        print 'shift', shift, 'estimator is ', estimator, 
        
        m = XGBRegressor(#n_jobs=8,
                         tree_method='gpu_hist', 
                         learning_rate=topRates[shift],
                         n_estimators = estimator)

        trainF = allData[train_from : train_to]
        fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

        mae = metrics.mean_absolute_error(allData[test_from : thttp://localhost:8888/notebooks/Nikolai_L_C6W6.ipynb#est_to]['target_' + str(shift)],
                                          fitted.predict(allData[test_from : test_to][x_names]))
        
        if mae < bestMae:
            bestMae = mae
            bestEstimator = estimator
        
        print 'MAE',  mae

    topEstimators[shift] = bestEstimator
    print 'shift', shift, 'best estimator is ', bestEstimator, 'MAE', bestMae
#15.545795647531866

shift 1 estimator is  100 MAE 18.900434418162554
shift 1 estimator is  200 MAE 17.83322728108394
shift 1 estimator is  300 MAE 17.250590258158525
shift 1 estimator is  400 MAE 16.922877957842623
shift 1 best estimator is  400 MAE 16.922877957842623
shift 2 estimator is  100 MAE 23.363585709830843
shift 2 estimator is  200 MAE 22.051440028749752
shift 2 estimator is  300 MAE 21.349268957645158
shift 2 estimator is  400 MAE 20.83652849644901
shift 2 best estimator is  400 MAE 20.83652849644901
shift 3 estimator is  100 MAE 24.885318830423497
shift 3 estimator is  200 MAE 23.469912973830642
shift 3 estimator is  300 MAE 22.714594982633045
shift 3 estimator is  400 MAE 22.209271187177954
shift 3 best estimator is  400 MAE 22.209271187177954
shift 4 estimator is  100 MAE 25.897806017240733
shift 4 estimator is  200 MAE 24.448874907419782
shift 4 estimator is  300 MAE 23.72192878486072
shift 4 estimator is  400 MAE 23.193369635875786
shift 4 best estimator is  400 MAE 23.193369635875786
shif

In [30]:
topEstimators

{1: 400, 2: 400, 3: 400, 4: 400, 5: 400, 6: 400}

In [8]:
topEstimators = {1: 400, 2: 400, 3: 400, 4: 400, 5: 400, 6: 400}

Подберем max_depth

In [32]:
depths = np.arange(5, 12, 1)

topDepths = {}        

for shift in range(1, nModels + 1):
    bestDepth = 0
    bestMae = sys.maxint

    for depth in depths:

        print 'shift', shift, 'depth is ', depth, 
        
        m = XGBRegressor(#n_jobs=8, 
                         tree_method='gpu_hist', 
                         learning_rate=topRates[shift],
                         n_estimators = topEstimators[shift],
                         max_depth = depth)

        trainF = allData[train_from : train_to]
        fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

        mae = metrics.mean_absolute_error(allData[test_from : test_to]['target_' + str(shift)],
                                          fitted.predict(allData[test_from : test_to][x_names]))
        
        if mae < bestMae:
            bestMae = mae
            bestDepth = depth
        
        print 'MAE',  mae

    topDepths[shift] = bestDepth
    print 'shift', shift, 'best depth is ', bestDepth, 'MAE', bestMae


 shift 1 depth is  5 MAE 16.157316477441814
shift 1 depth is  6 MAE 16.185746552153343
shift 1 depth is  7 MAE 16.147490041742383
shift 1 depth is  8 MAE 16.387686240815082
shift 1 depth is  9 MAE 16.513622348667766
shift 1 depth is  10 MAE 16.479866250964754
shift 1 depth is  11 MAE 16.59564050986164
shift 1 best depth is  7 MAE 16.147490041742383
shift 2 depth is  5 MAE 18.926378187616585
shift 2 depth is  6 MAE 18.7547874930902
shift 2 depth is  7 MAE 18.382574131802315
shift 2 depth is  8 MAE 18.33640459805475
shift 2 depth is  9 MAE 18.317950813331876
shift 2 depth is  10 MAE 18.296379156103292
shift 2 depth is  11 MAE 18.331924700023592
shift 2 best depth is  10 MAE 18.296379156103292
shift 3 depth is  5 MAE 20.71135648141807
shift 3 depth is  6 MAE 20.091156783654206
shift 3 depth is  7 MAE 19.759318863596086
shift 3 depth is  8 MAE 19.92640789105969
shift 3 depth is  9 MAE 19.90241657290269
shift 3 depth is  10 MAE 19.64022964161707
shift 3 depth is  11 MAE 19.762560654085927
s

In [34]:
topDepths

{1: 7, 2: 10, 3: 10, 4: 10, 5: 10, 6: 9}

In [9]:
topDepths = {1: 400, 2: 400, 3: 400, 4: 400, 5: 400, 6: 400}

Подберем gamma

In [35]:
gammas = np.arange(0, 130, 10)

topGammas = {}        

for shift in range(1, nModels + 1):
    bestGamma = 0
    bestMae = sys.maxint

    for gamma in gammas:

        print 'shift', shift, 'gamma is ', gamma, 
        
        m = XGBRegressor(#n_jobs=8, 
                         tree_method='gpu_hist', 
                         learning_rate=topRates[shift],
                         n_estimators = topEstimators[shift],
                         max_depth = topDepths[shift],
                         gamma=gamma)

        trainF = allData[train_from : train_to]
        fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

        mae = metrics.mean_absolute_error(allData[test_from : test_to]['target_' + str(shift)],
                                          fitted.predict(allData[test_from : test_to][x_names]))
        
        if mae < bestMae:
            bestMae = mae
            bestGamma = gamma
        
        print 'MAE',  mae

    topGammas[shift] = bestGamma
    print 'shift', shift, 'best gamma is ', bestGamma, 'MAE', bestMae


shift 1 gamma is  0 MAE 16.225128123188533
shift 1 gamma is  10 MAE 16.22453306922319
shift 1 gamma is  20 MAE 16.22512910662524
shift 1 gamma is  30 MAE 16.22157826225232
shift 1 gamma is  40 MAE 16.224531889042844
shift 1 gamma is  50 MAE 16.221578618786587
shift 1 gamma is  60 MAE 16.2245318482484
shift 1 gamma is  70 MAE 16.221578864897516
shift 1 gamma is  80 MAE 16.22157900117065
shift 1 gamma is  90 MAE 16.22157927379283
shift 1 gamma is  100 MAE 16.23047220754898
shift 1 gamma is  110 MAE 16.22157881626562
shift 1 gamma is  120 MAE 16.22157845224143
shift 1 best gamma is  30 MAE 16.22157826225232
shift 2 gamma is  0 MAE 18.327297569788847
shift 2 gamma is  10 MAE 18.312218312138796
shift 2 gamma is  20 MAE 18.315099372259862
shift 2 gamma is  30 MAE 18.30637318142957
shift 2 gamma is  40 MAE 18.325102365950173
shift 2 gamma is  50 MAE 18.313138681482208
shift 2 gamma is  60 MAE 18.32397883105161
shift 2 gamma is  70 MAE 18.324181767759338
shift 2 gamma is  80 MAE 18.28721947556

In [36]:
topGammas

{1: 30, 2: 100, 3: 90, 4: 0, 5: 40, 6: 30}

In [10]:
topGammas = {1: 30, 2: 100, 3: 90, 4: 0, 5: 40, 6: 30}

Подберем subsample

In [ ]:
subsamples = np.linspace(0.9, 1, 10)

topSubsamples = {}        
topModels = {}   

for shift in range(1, nModels + 1):
    bestSubsample = 0
    bestMae = sys.maxint

    for subsample in subsamples:

        print 'shift', shift, 'ssample is ', subsample, 
        
        m = XGBRegressor(#n_jobs=8, 
                         tree_method='gpu_hist', 
                         learning_rate=topRates[shift],
                         n_estimators = topEstimators[shift],
                         max_depth = topDepths[shift],
                         gamma=topGammas[shift],
                         subsample = subsample)

        trainF = allData[train_from : train_to]
        fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

        mae = metrics.mean_absolute_error(allData[test_from : test_to]['target_' + str(shift)],
                                          fitted.predict(allData[test_from : test_to][x_names]))
        
        if mae < bestMae:
            bestMae = mae
            bestSubsample = subsample
            topModels[shift] = fitted
        print 'MAE',  mae

    topSubsamples[shift] = bestSubsample
    print 'shift', shift, 'best ssample is ', bestSubsample, 'MAE', bestMae


shift 1 ssample is  0.9

In [38]:
topSubsamples

{1: 0.9894736842105263,
 2: 1.0,
 3: 0.9947368421052631,
 4: 0.968421052631579,
 5: 0.9842105263157894,
 6: 0.9842105263157894}

In [11]:
topSubsamples = {1: 0.9894736842105263,
 2: 1.0,
 3: 0.9947368421052631,
 4: 0.968421052631579,
 5: 0.9842105263157894,
 6: 0.9842105263157894}

Обучим итоговые модели с подобранным параметрами 

In [14]:
topModels = {}        

for shift in range(1, nModels + 1):

    m = XGBRegressor(n_jobs=8, 
                     #tree_method='gpu_hist', 
                     learning_rate=topRates[shift],
                     n_estimators = topEstimators[shift],
                     max_depth = topDepths[shift],
                     gamma=topGammas[shift],
                     subsample = topSubsamples[shift])

    trainF = allData[train_from : train_to]
    fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

    mae = metrics.mean_absolute_error(allData[test_from : test_to]['target_' + str(shift)],
                                      fitted.predict(allData[test_from : test_to][x_names]))

    topModels[shift] = fitted
    
    print 'shift', shift, 'MAE', mae


shift 1 MAE 17.294633870845388
shift 2 MAE 18.849983362054047
shift 3 MAE 20.448574746076368
shift 4 MAE 21.77659240675474
shift 5 MAE 22.060539293350548
shift 6 MAE 21.915363930641597


Метрика за май

In [15]:
totalMay = 0

for shift in range(1, nModels + 1):
    
    fitted = topModels[shift]

    mae = metrics.mean_absolute_error(allData[test_from : test_to]['target_' + str(shift)],
                                      fitted.predict(allData[test_from : test_to][x_names]))
    print mae
    totalMay += mae   


17.294633870845388
18.849983362054047
20.448574746076368
21.77659240675474
22.060539293350548
21.915363930641597


In [16]:
Qmay = totalMay / 6
print Qmay

20.390947934953783


Предыдущее значение 31.233553749418448. Есть улучшения.

Обучим модели на данных до конца мая

In [12]:
train_from = pd.to_datetime('2016-01-01 00:00:00')
train_to = pd.to_datetime('2016-05-31 17:00:00')

test_from = pd.to_datetime('2016-05-31 23:00:00')
test_to = pd.to_datetime('2016-06-30 17:00:00')

topModels = {}

for shift in range(1, nModels + 1):
    m = XGBRegressor(n_jobs=8, 
                     #tree_method='gpu_hist', 
                     learning_rate=topRates[shift],
                     n_estimators = topEstimators[shift],
                     max_depth = topDepths[shift],
                     gamma=topGammas[shift],
                     subsample = topSubsamples[shift])

    trainF = allData[train_from : train_to]
    fitted = m.fit(trainF[x_names], trainF['target_' + str(shift)])

    mae = metrics.mean_absolute_error(allData[test_from : test_to]['target_' + str(shift)],
                                      fitted.predict(allData[test_from : test_to][x_names]))

    topModels[shift] = fitted
    print 'shift', shift, 'MAE', mae


shift 1 MAE 16.724215643848556
shift 2 MAE 17.879316417873518
shift 3 MAE 18.995883797810563
shift 4 MAE 20.373902338450623
shift 5 MAE 20.688182976086345
shift 6 MAE 20.81693647300512


Метрика за июнь

In [13]:
totalJune = 0

for shift in range(1, nModels + 1):
    
    fitted = topModels[shift]

    mae = metrics.mean_absolute_error(allData[test_from : test_to]['target_' + str(shift)],
                                      fitted.predict(allData[test_from : test_to][x_names]))
    print mae
    totalJune += mae   


16.724215643848556
17.879316417873518
18.995883797810563
20.373902338450623
20.688182976086345
20.81693647300512


In [14]:
Qjune = totalJune / 6
print Qjune #17.64

19.246406274512452


Предыдущее значение 29.45654190596134. Есть улучшения.

Запишем результаты в файл

In [15]:
junePred = {}
for reg in data.region.unique():
    #for ix in range(0, 715):
    print reg,
    regPred = {}
    for shift in range(1, nModels + 1):
        
        fitted = topModels[shift]

        pred = fitted.predict(allData[allData.region == reg][test_from:test_to][x_names])  

        
        regPred[shift] = pred
    junePred[reg] = regPred

1075 1076 1077 1125 1126 1127 1128 1129 1130 1131 1132 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533 1580 1630 1684 1733 1734 1783 2068 2069 2118 2119 2168


In [17]:
results = []
for reg in junePred:
    print reg,
    regPred = junePred[reg]
    for shift in range(1, nModels + 1):
        pred = regPred[shift]
        for ix in range(0, 715):
            tPred = test_from + datetime.timedelta(hours=ix)
            pdId = str.format("{0}_{1}_{2}_{3}",reg, tPred.date(), tPred.hour, shift)
            results.append((pdId, pred[ix]))
    


2068 2069 1580 1075 1076 1077 2118 2119 1630 1125 1126 1127 1128 1129 1130 1131 1132 2168 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1734 1221 1222 1223 1224 1225 1227 1228 1229 1230 1231 1232 1233 1234 1235 1783 1272 1273 1274 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1326 1327 1331 1332 1333 1334 1335 1336 1337 1338 1339 1733 1376 1377 1378 1380 1382 1383 1384 1385 1386 1387 1388 1389 1390 1684 1426 1431 1434 1435 1436 1437 1438 1439 1441 1442 1480 1482 1483 1530 1532 1533


In [20]:
juneDf = pd.DataFrame(results)

In [21]:
juneDf.to_csv('TaxiData/w6_kaggle.csv',index=False)

Ссылка https://www.kaggle.com/submissions/7749080/7749080.zip